# Truck Driver Routing Scheduling

This is a rephrasing of the nurse scheduling problem from Google OR Tools.
<br>
#### The problem
A logistics company needs to schedule four truck drivers for deliveries over a three-day period. <br>
Each day is split into three main delivery windows (morning, afternoon, evening). <br>
Every delivery window must be covered by one driver, and no driver can cover more than one window per day. <br>
To maintain work-life balance, each driver must be assigned to at least two delivery windows over the three days.
<br> __________________________________________________________________________________________
<br>
<br>
<b> We add a new constraint which is more realistic </b>
<br>
A human truck driver cannot work in the morning the day after if they had the evening shift the day before. <br>
In some countries this is both illegal according to law or labour unions forbids it.

In [13]:
from ortools.sat.python import cp_model

In [2]:
num_drivers = 4
num_windows = 3
num_days = 3
all_drivers = range(num_drivers)
all_windows = range(num_windows)
all_days = range(num_days)

model = cp_model.CpModel()

In [3]:
# Create boolean variables for assignments
assignments = {}
for d in all_drivers:
    for day in all_days:
        for w in all_windows:
            assignments[(d, day, w)] = model.new_bool_var(f"assignment_d{d}_day{day}_w{w}")


In [4]:
# Constraint: Each delivery window must be covered by one driver
for day in all_days:
    for w in all_windows:
        model.add_exactly_one(assignments[(d, day, w)] for d in all_drivers)


In [5]:
# Constraint: No driver can cover more than one window per day
for d in all_drivers:
    for day in all_days:
        model.add_at_most_one(assignments[(d, day, w)] for w in all_windows)


In [6]:
# Constraint: Each driver must be assigned at least two delivery windows over the three days
min_windows_per_driver = (num_windows * num_days) // num_drivers
if num_windows * num_days % num_drivers == 0:
    max_windows_per_driver = min_windows_per_driver
else:
    max_windows_per_driver = min_windows_per_driver + 1


In [7]:
for d in all_drivers:
    windows_worked = []
    for day in all_days:
        for w in all_windows:
            windows_worked.append(assignments[(d, day, w)])
    model.add(min_windows_per_driver <= sum(windows_worked))
    model.add(sum(windows_worked) <= max_windows_per_driver)


In [8]:
# New constraint: If a driver had window 2 on day t-1, they cannot have window 0 on day t
for d in all_drivers:
    for day in range(1, num_days):
        model.add(assignments[(d, day-1, 2)] + assignments[(d, day, 0)] <= 1)


In [9]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solver.parameters.enumerate_all_solutions = True


In [10]:
class DriverSchedulePrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, assignments, num_drivers, num_days, num_windows, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._assignments = assignments
        self._num_drivers = num_drivers
        self._num_days = num_days
        self._num_windows = num_windows
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        for day in range(self._num_days):
            print(f"Day {day}")
            for d in range(self._num_drivers):
                is_working = False
                for w in range(self._num_windows):
                    if self.value(self._assignments[(d, day, w)]):
                        is_working = True
                        print(f"  Driver {d} covers delivery window {w}")
                if not is_working:
                    print(f"  Driver {d} is off duty")
        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.stop_search()

    def solution_count(self):
        return self._solution_count

In [11]:
# Display the first five solutions
solution_limit = 5
solution_printer = DriverSchedulePrinter(
    assignments, num_drivers, num_days, num_windows, solution_limit
)


In [12]:
solver.solve(model, solution_printer)

Solution 1
Day 0
  Driver 0 is off duty
  Driver 1 covers delivery window 1
  Driver 2 covers delivery window 2
  Driver 3 covers delivery window 0
Day 1
  Driver 0 covers delivery window 2
  Driver 1 is off duty
  Driver 2 covers delivery window 1
  Driver 3 covers delivery window 0
Day 2
  Driver 0 covers delivery window 2
  Driver 1 covers delivery window 1
  Driver 2 covers delivery window 0
  Driver 3 is off duty
Solution 2
Day 0
  Driver 0 covers delivery window 1
  Driver 1 is off duty
  Driver 2 covers delivery window 2
  Driver 3 covers delivery window 0
Day 1
  Driver 0 is off duty
  Driver 1 covers delivery window 2
  Driver 2 covers delivery window 1
  Driver 3 covers delivery window 0
Day 2
  Driver 0 covers delivery window 2
  Driver 1 covers delivery window 1
  Driver 2 covers delivery window 0
  Driver 3 is off duty
Solution 3
Day 0
  Driver 0 covers delivery window 1
  Driver 1 is off duty
  Driver 2 covers delivery window 2
  Driver 3 covers delivery window 0
Day 1
  

2